In [1]:
import pandas as pd

# Load the dataset
file_path = '合併資料.csv'
data = pd.read_csv(file_path)

# Display the first few rows of the dataframe
data.head()


,日期,1101,1102,1103,1104,1108,1109,1110,1201,1203,...,市場風險溢酬,規模溢酬 (5因子),淨值市價比溢酬,益本比溢酬,股利殖利率溢酬,動能因子,短期反轉因子,長期反轉因子,盈利能力因子,投資因子
0,2024/5/1,5.9284,-0.0002,-1.3584,-4.6944,-0.2951,-1.9999,0.0001,-1.5831,3.3031,...,0.2691,1.9981,3.8834,-2.2549,-1.2096,-1.4161,-3.0940,0.7046,-3.4818,-1.9009
1,2024/4/1,-0.3112,5.8113,5.7471,11.2343,7.2786,4.1667,13.1654,0.2645,-1.4791,...,6.6041,-6.4691,1.6685,6.9829,1.2161,3.3173,-3.6908,-3.6969,6.6874,-4.4059
2,2024/3/1,0.1555,2.1014,-1.6950,6.3976,0.3177,7.5632,-0.2793,0.0000,-6.8014,...,5.7585,2.7441,-1.1266,2.1906,0.0219,6.3394,-5.6515,-3.4706,2.9705,-1.1088
3,2024/2/1,0.6269,1.7610,-0.8402,1.8867,3.6185,0.8474,-1.3773,1.3403,-1.9818,...,-0.1377,1.8310,0.1511,2.8331,-2.2041,6.1843,-3.1276,-3.4922,1.9416,-3.0190
4,2024/1/1,-8.4647,-4.1014,-6.2994,-1.8517,-3.7973,-3.0137,-3.4574,-2.0998,-2.4604,...,2.7020,-1.5516,2.1723,0.5634,1.8310,0.0293,-0.4815,-2.0000,0.2363,1.6656


In [8]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.neural_network import MLPRegressor
from statsmodels.api import OLS, add_constant
import numpy as np

# Drop the '日期' column for regression
data = data.drop(columns=['日期'])

# Separate X (predictors) and Y (responses)
X = data[['市場風險溢酬', '規模溢酬 (5因子)', '淨值市價比溢酬', '益本比溢酬', '股利殖利率溢酬', '動能因子', '短期反轉因子', '長期反轉因子', '盈利能力因子', '投資因子']]
Y = data.drop(columns=X.columns)

X.replace('-', np.nan, inplace=True)
Y.replace('-', np.nan, inplace=True)

# 将所有列转换为浮点数类型
X = X.apply(pd.to_numeric, errors='coerce')
Y = Y.apply(pd.to_numeric, errors='coerce')


C:\Users\asas9\AppData\Local\Temp\ipykernel_14068\3121680129.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.replace('-', np.nan, inplace=True)


In [9]:
# Store results
results = []

# Loop through each stock
for stock in Y.columns:
    y = Y[stock]
    
    # Handle missing data by dropping rows with NaN values
    df = pd.concat([X, y], axis=1).dropna()
    if df.empty or df[y.name].nunique() <= 1:
        continue  # Skip if the data is empty or has no variance
    
    X_clean = df[X.columns]
    y_clean = df[y.name]
    
    # Linear Regression
    lm = LinearRegression()
    lm.fit(X_clean, y_clean)
    y_pred_lm = lm.predict(X_clean)
    rmse_lm = mean_squared_error(y_clean, y_pred_lm, squared=False)
    
    # Stepwise Regression
    sfs = SequentialFeatureSelector(lm, n_features_to_select="auto", direction='forward')
    sfs.fit(X_clean, y_clean)
    X_sfs = X_clean.loc[:, sfs.get_support()]
    lm.fit(X_sfs, y_clean)
    y_pred_sfs = lm.predict(X_sfs)
    rmse_sfs = mean_squared_error(y_clean, y_pred_sfs, squared=False)
    
    # Neural Network
    nn = MLPRegressor(hidden_layer_sizes=(10,), max_iter=1000, random_state=0)
    nn.fit(X_clean, y_clean)
    y_pred_nn = nn.predict(X_clean)
    rmse_nn = mean_squared_error(y_clean, y_pred_nn, squared=False)
    
    # Evaluate model performance
    best_rmse = min(rmse_lm, rmse_sfs, rmse_nn)
    best_model = 'lm' if best_rmse == rmse_lm else 'sfs' if best_rmse == rmse_sfs else 'nn'
    
    # Check intercept significance using OLS
    ols_model = OLS(y_clean, add_constant(X_clean)).fit()
    intercept_pvalue = ols_model.pvalues[0]
    
    if intercept_pvalue < 0.05:
        results.append((stock, best_model, best_rmse, intercept_pvalue, ols_model.summary()))

results.sort(key=lambda x: x[2])  # Sort by RMSE

results[:5]  # Display top 5 results


C:\Users\asas9\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\asas9\AppData\Local\Temp\ipykernel_14068\2337812038.py:42: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  intercept_pvalue = ols_model.pvalues[0]
C:\Users\asas9\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\asas9\AppData\Local\Temp\ipykernel_14068\2337812038.py:42: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be

C:\Users\asas9\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\asas9\AppData\Local\Temp\ipykernel_14068\2337812038.py:42: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  intercept_pvalue = ols_model.pvalues[0]
C:\Users\asas9\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\asas9\AppData\Local\Temp\ipykernel_14068\2337812038.py:42: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be

C:\Users\asas9\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\asas9\AppData\Local\Temp\ipykernel_14068\2337812038.py:42: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  intercept_pvalue = ols_model.pvalues[0]
C:\Users\asas9\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\asas9\AppData\Local\Temp\ipykernel_14068\2337812038.py:42: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be

C:\Users\asas9\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\asas9\AppData\Local\Temp\ipykernel_14068\2337812038.py:42: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  intercept_pvalue = ols_model.pvalues[0]
C:\Users\asas9\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\asas9\AppData\Local\Temp\ipykernel_14068\2337812038.py:42: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be

C:\Users\asas9\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\asas9\AppData\Local\Temp\ipykernel_14068\2337812038.py:42: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  intercept_pvalue = ols_model.pvalues[0]
C:\Users\asas9\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\asas9\AppData\Local\Temp\ipykernel_14068\2337812038.py:42: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be

...

C:\Users\asas9\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\asas9\AppData\Local\Temp\ipykernel_14068\2337812038.py:42: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  intercept_pvalue = ols_model.pvalues[0]
C:\Users\asas9\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\asas9\AppData\Local\Temp\ipykernel_14068\2337812038.py:42: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be

C:\Users\asas9\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\asas9\AppData\Local\Temp\ipykernel_14068\2337812038.py:42: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  intercept_pvalue = ols_model.pvalues[0]
C:\Users\asas9\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\asas9\AppData\Local\Temp\ipykernel_14068\2337812038.py:42: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be

C:\Users\asas9\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\asas9\AppData\Local\Temp\ipykernel_14068\2337812038.py:42: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  intercept_pvalue = ols_model.pvalues[0]
C:\Users\asas9\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\asas9\AppData\Local\Temp\ipykernel_14068\2337812038.py:42: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be

C:\Users\asas9\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\asas9\AppData\Local\Temp\ipykernel_14068\2337812038.py:42: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  intercept_pvalue = ols_model.pvalues[0]
C:\Users\asas9\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\asas9\AppData\Local\Temp\ipykernel_14068\2337812038.py:42: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be

[('6184',
  'nn',
  1.3420514234111869,
  0.00011890546827717608,
  <class 'statsmodels.iolib.summary.Summary'>
  """
                              OLS Regression Results                            
  Dep. Variable:                   6184   R-squared:                       0.180
  Model:                            OLS   Adj. R-squared:                  0.045
  Method:                 Least Squares   F-statistic:                     1.336
  Date:                Mon, 03 Jun 2024   Prob (F-statistic):              0.232
  Time:                        17:18:01   Log-Likelihood:                -151.42
  No. Observations:                  72   AIC:                             324.8
  Df Residuals:                      61   BIC:                             349.9
  Df Model:                          10                                         
  Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025    